In [ ]:
from ultralytics import YOLO
import cv2, os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


model = YOLO('yolov8n.pt')
video = cv2.VideoCapture(0)

while True:
    _, image = video.read()

    result = model.track(image, persist=True, tracker='bytetrack.yaml')
    image_ = result[0].plot()
    cv2.imshow('image', image_)
    if cv2.waitKey(1) == ord('o'): break


cv2.destroyAllWindows()
video.release()

In [ ]:
import numpy as np
from ultralytics import YOLO
import cv2, os
from collections import defaultdict

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

model = YOLO('yolov8n.pt')
video = cv2.VideoCapture(0)

track_history = defaultdict(lambda: [])



while True:
    _, image = video.read()

    result = model.track(image, persist=True)

    boxes = result[0].boxes.xywh.cpu()
    tracks_id = result[0].boxes.id.int().cpu().tolist()

    annotated_frame = result[0].plot()

    for box, track_id in zip(boxes, tracks_id):
        x, y, w, h = box
        track = track_history[track_id]
        track.append((float(x), float(y)))
        if len(track) > 30:
            track.pop(0)


        points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
        cv2.polylines(annotated_frame, [points], isClosed=False, color=(0, 0, 250), thickness=2)



    cv2.imshow('Tracking Yolo-v8', annotated_frame)
    if cv2.waitKey(1) == ord('o'): break


cv2.destroyAllWindows()
video.release()